# Connect Franka

In [8]:
# Panda hostname/IP and Desk login information of your robot
hostname = '172.16.0.2'
username = 'admin'
password = '******'

# panda-py is chatty, activate information log level
import logging
logging.basicConfig(level=logging.INFO)

In [9]:
import panda_py
from panda_py import libfranka

panda = panda_py.Panda(hostname)
gripper = libfranka.Gripper(hostname)
panda.move_to_start()

INFO:panda:Connected to robot (172.16.0.2).
INFO:panda:Panda class destructor invoked (172.16.0.2).
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 0.97 seconds.
INFO:panda:Starting new controller (Trajectory).


True

In [10]:
from panda_py import constants
from scipy.spatial.transform import Rotation as R
import numpy as np
import matplotlib.pyplot as plt
import os
import time

# Shoot from Different Views

In [17]:
# read shooting pose(7 dof joint position in one line) from file
joint_position_list = np.loadtxt("shooting_pose_tabletop.txt", delimiter=',')

In [18]:
import sys
parent_dir = os.path.dirname(os.getcwd())
parent_dir = os.path.dirname(parent_dir)
sys.path.append(parent_dir)
from realsense.realsense import Camera


base_dir = '/home/yisha/workspace/colmap_handeye/video'
os.makedirs(f'{base_dir}/images', exist_ok=True)
os.makedirs(f'{base_dir}/depth', exist_ok=True)
os.makedirs(f'{base_dir}/poses', exist_ok=True)

device_serial = '233622078546' # '145422070656'

rgb_resolution = (1280, 720)  # RGB resolution (width, height)
depth_resolution = (1280, 720)  # Depth resolution (width, height)
camera = Camera(device_serial, rgb_resolution, depth_resolution)
camera.start()

rgb_intrinsics, rgb_coeffs, depth_intrinsics, depth_coeffs = camera.get_intrinsics_raw()
depth_scale = camera.get_depth_scale()

print(f"RGB Intrinsics: {rgb_intrinsics}")
print(f"RGB Distortion Coefficients: {rgb_coeffs}")
rgb_intrinsics_path = f'{base_dir}/rgb_intrinsics.npz'
np.savez(rgb_intrinsics_path, fx=rgb_intrinsics.fx, fy=rgb_intrinsics.fy, ppx=rgb_intrinsics.ppx, ppy=rgb_intrinsics.ppy, coeffs=rgb_intrinsics.coeffs)

print(f"Depth Scale: {depth_scale}")
print(f"Depth Intrinsics: {depth_intrinsics}")
print(f"Depth Distortion Coefficients: {depth_coeffs}")
depth_intrinsics_path = f'{base_dir}/depth_intrinsics.npz'
np.savez(depth_intrinsics_path, fx=depth_intrinsics.fx, fy=depth_intrinsics.fy, ppx=depth_intrinsics.ppx, ppy=depth_intrinsics.ppy, coeffs=depth_intrinsics.coeffs, depth_scale=depth_scale)

# drop the first few frames to allow the camera to warm up
_, _ = camera.shoot()  
time.sleep(2)
panda.move_to_start()

for i in range(joint_position_list.shape[0]):
    joint_position = joint_position_list[i]
    panda.move_to_joint_position(joint_position)
    pose = panda.get_pose()

    rgb_image, depth_image = camera.shoot()
    depth_image = depth_image * depth_scale
    rgb_filename = f'{base_dir}/images/{i}.png'
    depth_filename = f'{base_dir}/depth/{i}.npy'
    plt.imsave(rgb_filename, rgb_image)
    np.save(depth_filename, depth_image)
    print(f"Saved {rgb_filename}")
    print(f"Saved {depth_filename}")

    def save_pose(pose, base_dir, frame_num):
        """Save the robot arm's pose to a file."""
        pose_filename = f'{base_dir}/poses/{frame_num}.npy'
        np.save(pose_filename, pose)
        print(f"Saved pose to {pose_filename}")

    save_pose(pose, base_dir, i)

RGB Intrinsics: [ 1280x720  p[633.573 361.852]  f[909.08 908.376]  Inverse Brown Conrady [0 0 0 0 0] ]
RGB Distortion Coefficients: [0.0, 0.0, 0.0, 0.0, 0.0]
Depth Scale: 0.0010000000474974513
Depth Intrinsics: [ 1280x720  p[643.049 358.826]  f[646.826 646.826]  Brown Conrady [0 0 0 0 0] ]
Depth Distortion Coefficients: [0.0, 0.0, 0.0, 0.0, 0.0]


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 0.97 seconds.
INFO:panda:Starting new controller (Trajectory).
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 1.86 seconds.
INFO:panda:Starting new controller (Trajectory).
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 2.76 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved /home/yisha/workspace/colmap_handeye/video/images/0.png
Saved /home/yisha/workspace/colmap_handeye/video/depth/0.npy
Saved pose to /home/yisha/workspace/colmap_handeye/video/poses/0.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 1.60 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved /home/yisha/workspace/colmap_handeye/video/images/1.png
Saved /home/yisha/workspace/colmap_handeye/video/depth/1.npy
Saved pose to /home/yisha/workspace/colmap_handeye/video/poses/1.npy


...

Saved /home/yisha/workspace/colmap_handeye/video/images/19.png
Saved /home/yisha/workspace/colmap_handeye/video/depth/19.npy
Saved pose to /home/yisha/workspace/colmap_handeye/video/poses/19.npy


ERROR:panda:Control loop interruped: libfranka: Move command aborted: motion aborted by reflex! ["max_path_pose_deviation_violation"]
control_command_success_rate: 1
INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 1.67 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved /home/yisha/workspace/colmap_handeye/video/images/20.png
Saved /home/yisha/workspace/colmap_handeye/video/depth/20.npy
Saved pose to /home/yisha/workspace/colmap_handeye/video/poses/20.npy


INFO:panda:Stopping active controller (Trajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 4.08 seconds.
INFO:panda:Starting new controller (Trajectory).


Saved /home/yisha/workspace/colmap_handeye/video/images/21.png
Saved /home/yisha/workspace/colmap_handeye/video/depth/21.npy
Saved pose to /home/yisha/workspace/colmap_handeye/video/poses/21.npy
Saved /home/yisha/workspace/colmap_handeye/video/images/22.png
Saved /home/yisha/workspace/colmap_handeye/video/depth/22.npy
Saved pose to /home/yisha/workspace/colmap_handeye/video/poses/22.npy


# Realsense Video Recording

In [13]:
# import sys
# parent_dir = os.path.dirname(os.getcwd())
# parent_dir = os.path.dirname(parent_dir)
# sys.path.append(parent_dir)
# from realsense.realsense import Camera


# base_dir = '/home/yisha/workspace/colmap_handeye/video'
# os.makedirs(f'{base_dir}/images', exist_ok=True)
# os.makedirs(f'{base_dir}/depth', exist_ok=True)
# os.makedirs(f'{base_dir}/poses', exist_ok=True)

# device_serial = '145422070656'

# frame_num = 10
# rgb_resolution = (1280, 720)  # RGB resolution (width, height)
# depth_resolution = (1280, 720)  # Depth resolution (width, height)
# camera = Camera(device_serial, rgb_resolution, depth_resolution)
# camera.start()

# rgb_intrinsics, rgb_coeffs, depth_intrinsics, depth_coeffs = camera.get_intrinsics_raw()
# depth_scale = camera.get_depth_scale()

# print(f"RGB Intrinsics: {rgb_intrinsics}")
# print(f"RGB Distortion Coefficients: {rgb_coeffs}")
# rgb_intrinsics_path = f'{base_dir}/rgb_intrinsics.npz'
# np.savez(rgb_intrinsics_path, fx=rgb_intrinsics.fx, fy=rgb_intrinsics.fy, ppx=rgb_intrinsics.ppx, ppy=rgb_intrinsics.ppy, coeffs=rgb_intrinsics.coeffs)

# print(f"Depth Scale: {depth_scale}\n")
# print(f"Depth Intrinsics: {depth_intrinsics}\n")
# print(f"fx:{depth_intrinsics.fx},fy:{depth_intrinsics.fy},cx:{depth_intrinsics.ppx},cy:{depth_intrinsics.ppy}")
# print(f"Depth Distortion Coefficients: {depth_coeffs}\n")
# depth_intrinsics_path = f'{base_dir}/depth_intrinsics.npz'
# np.savez(depth_intrinsics_path, fx=depth_intrinsics.fx, fy=depth_intrinsics.fy, ppx=depth_intrinsics.ppx, ppy=depth_intrinsics.ppy, coeffs=depth_intrinsics.coeffs, depth_scale=depth_scale)

# # drop the first few frames to allow the camera to warm up
# _, _ = camera.shoot()  
# time.sleep(2)
# for i in range(frame_num):
#     time.sleep(1)
#     rgb_image, depth_image = camera.shoot()
#     depth_image = depth_image * depth_scale
#     rgb_filename = f'{base_dir}/rgb/{i}.png'
#     # depth_filename = f'{base_dir}/depth/{i}.npy'
#     depth_filename = f'{base_dir}/depth/{i}.png'
#     plt.imsave(rgb_filename, rgb_image)
#     plt.imsave(depth_filename, depth_image)
#     # np.save(depth_filename, depth_image)
#     print(f"Saved {rgb_filename}")
#     print(f"Saved {depth_filename}")


In [14]:
# import cv2
# import numpy as np
# depth = cv2.imread('/home/yisha/workspace/colmap_handeye/mask.png', 0)
# _, binary_mask = cv2.threshold(depth, 0, 255, cv2.THRESH_BINARY)
# cv2.imwrite('mask.png', binary_mask)


In [15]:
# import cv2
# for i in range(10):
#     path_2_img = f'/home/yisha/workspace/colmap_handeye/video/rgb/{i}.png'
#     img = cv2.imread(path_2_img)
#     h, w = img.shape[:2]
#     cv2.imwrite(f'/home/yisha/workspace/FoundationPose/demo_data/mustard2/rgb/{i}.png', cv2.resize(img,(w // 2, h // 2)))
#     if i == 0:
#         cv2.imwrite(f'/home/yisha/workspace/sam2/notebooks/{i}.png', cv2.resize(img,(w // 2, h // 2)))

In [16]:
# for i in range(10):
#     # path_2_dep = f'/home/yisha/workspace/colmap_handeye/video/depth_resize/{i}.png'
#     # img = cv2.imread(path_2_dep)
#     # print(f'Max:{img.max()}')
#     # print(path_2_dep, img.shape,'\n')
#     path_2_img = f'/home/yisha/workspace/colmap_handeye/video/rgb_resize/{i}.png'
#     img = cv2.imread(path_2_img)
#     print(path_2_img, img.shape,'\n')